In [ ]:
import numpy as np
from numpy import sin, cos,exp,pi
from time import process_time, time
from scipy.optimize import curve_fit, least_squares
from scipy.fft import fft,fftshift,fft2, fftfreq, ifft
from scipy.signal import oaconvolve, medfilt2d

from scipy.ndimage import rotate
from scipy.interpolate import interp1d

# import h5py
from skimage.transform import radon
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib import transforms
from PIL import Image, ImageFilter
import joblib as jb
import tifffile as tfl
import os
from IPython.display import clear_output
import re
import pandas as pd

from astropy.io import fits

In [ ]:
%run "C:\Users\gabri\Documents\Python Scripts\NPGMI-Library\NPGMI Base Code.ipynb"

### 2D Grating Data Analysis

In [ ]:
t0 = time()

exp2path = r"C:\Users\gabri\Documents\Python Scripts\JPARC experiment\exp2"




exp2polyfiltered = np.array([[process_tfl(img, -7.8, 5)[20:image_side - 100, 20:image_side - 100] for img in np.array(tfl.imread(exp2path  + "\\" + path))]\
                         for i, path in enumerate(os.listdir(exp2path)[-2:-1])]).squeeze()

print(time() - t0, "seconds")

In [ ]:
poly2ddataIx = np.sum(exp2filtered, axis = -2)
poly2ddataIy = np.sum(exp2filtered, axis = -1)
poly2dx = np.arange(poly2ddataIx.shape[-1])*cm_per_pixel*1e-2
poly2dy = np.arange(poly2ddataIy.shape[-1])*cm_per_pixel*1e-2

# pxdata, pydata = 2.97e-6, 2.9e-6
pxdata, pydata = 3e-6, 3e-6

Ldata = 8.2955 
# Ldata = 8.352789
d02d = 0
d2data = np.linspace(4.5,18.5,8)*1e-3 - d02d



In [ ]:
# exp2monofiltered = np.array([process_tfl(img,14,5) for img in tfl.imread(exp2path  + "\\" + "NID019772_20230217.tiff")])


exp2mono = np.array([process_tfl(img[...,275:875,200:850],14,5,filter_flag=False) for img in tfl.imread(exp2path  + "\\" + "NID019772_20230217.tiff")])

In [ ]:
mono2ddataIx = np.sum(exp2mono, axis = -2)
mono2ddataIy = np.sum(exp2mono, axis = -1)
mono2dx = np.arange(mono2ddataIx.shape[-1])*cm_per_pixel
mono2dy = np.arange(mono2ddataIy.shape[-1])*cm_per_pixel


In [ ]:
poly2ddataIxfitparams = np.array([best_fit_moire_period(cosine_func, poly2dx, poly2ddataIx[i], pxdata, Ldata, d2data[i]) for i in range(len(d2data))])
poly2ddataIyfitparams = np.array([best_fit_moire_period(cosine_func, poly2dy, poly2ddataIy[i], pydata, Ldata, d2data[i]) for i in range(len(d2data))])

# poly2ddataIxfitparams = np.array([curve_fit(cosine_func,poly2dx, poly2ddataIx[i],\
#                         p0 = [np.mean(poly2ddataIx[i]), np.max(poly2ddataIx[i]) - np.mean(poly2ddataIx[i]),Ldata*pxdata/d2data[i],0],\
#                         maxfev = int(3e4))[0] for i in range(len(d2data))])
# poly2ddataIyfitparams = np.array([curve_fit(cosine_func,poly2dy, poly2ddataIy[i],\
#                         p0 = [np.mean(poly2ddataIy[i]), np.max(poly2ddataIy[i]) - np.mean(poly2ddataIy[i]),Ldata*pydata/d2data[i],0],\
#                         maxfev = int(3e4))[0] for i in range(len(d2data))])



# mono2ddataIxfitparams = np.array([best_fit_moire_period(cosine_func, mono2dx, mono2ddataIx[i], pxdata, Ldata, d2data[i]) for i in range(len(d2data))])
# mono2ddataIyfitparams = np.array([best_fit_moire_period(cosine_func, mono2dy, mono2ddataIy[i], pydata, Ldata, d2data[i]) for i in range(len(d2data))])
# incorrect currently

In [ ]:
fig, ax = plt.subplots(figsize = (12,12))
ax.imshow(exp2polyfiltered[5])
ax.axvline(860)
fig.savefig("exp2polyfiltered.jpg")
plt.show()
plt.close(fig)

In [ ]:

exp2polycontx = np.abs(poly2ddataIxfitparams[...,1]/poly2ddataIxfitparams[...,0])
exp2polyconty = np.abs(poly2ddataIyfitparams[...,1]/poly2ddataIyfitparams[...,0])

exp2polyfreqx = 1/poly2ddataIxfitparams[...,2]
exp2polyfreqy = 1/poly2ddataIyfitparams[...,2]

In [ ]:
fig, ax1 = plt.subplots(figsize = (12,8))

ind = 2
# ax1.plot(poly2dy,poly2ddataIy[ind], label = "Data Along y")
ax1.plot(poly2dx,poly2ddataIx[ind], label = "Data Along x")

ax1.set_prop_cycle(None)

# ax1.plot(poly2dy,cosine_func(poly2dy,*poly2ddataIyfitparams[ind]), "--",label = "Fit Along y")
ax1.plot(poly2dx,cosine_func(poly2dx,*poly2ddataIxfitparams[ind]), "--",label = "Fit Along x")


ax1.set_xlabel("Camera x-axis (cm)")
ax1.legend()

# fig.savefig("exp2polyIydatawithfit.jpg")

plt.show()
plt.close(fig)

In [ ]:
fig, ax1 = plt.subplots(figsize = (12,12))

# ax1.imshow(exp2mono[36])

ax1.plot(mono2dy,mono2ddataIy[36], label = "Along y")
ax1.plot(mono2dx,mono2ddataIx[36], label = "Along x")

ax1.set_prop_cycle(None)

ax1.plot(mono2dy,cosine_func(mono2dy,*mono2ddataIyfitparams[36]), "--",label = "Fit Along y")
ax1.plot(mono2dx,cosine_func(mono2dx,*mono2ddataIxfitparams[36]), "--",label = "Fit Along x")


ax1.legend()
# fig.savefig("exp2monoIydata.jpg")
plt.show()
plt.close(fig)

In [ ]:
fig, ax1 = plt.subplots(figsize = (12,8))
ax1.plot(d2data*1e3, exp2polycontx*1e2, "x", label = "Along x")
ax1.plot(d2data*1e3, exp2polyconty*1e2, "x", label = "Along y")

ax1.set_xlabel("d (mm)")
ax1.set_ylabel("Contrast (%)")
ax1.set_title("2D Gratings Contrast vs. d")
ax1.legend()
# fig.savefig("exp2contvd.jpg")
plt.show()
plt.close(fig)

In [ ]:
fig, ax1 = plt.subplots(figsize = (12,8))
ax1.plot(d2data*1e3, exp2polyfreqx/1e3, "x", label = "Along x")
ax1.plot(d2data*1e3, exp2polyfreqy/1e3, "x", label = "Along y")

ax1.set_xlabel("d (mm)")
ax1.set_ylabel(r"Frequency $(mm^{-1})$")
ax1.set_title("2D Gratings Frequency vs. d")
ax1.legend()
# fig.savefig("exp2freqvd.jpg")
plt.show()
plt.close(fig)

### Simulations

In [ ]:
px, py = 3e-6, 3e-6

Nbc = 5e28*4.149e-15
mt = 5
x0 = 0


L1mono = 1.2

L0mono = -0.05

Lmono = 2.99 - L0mono


# polyspectrum = pd.read_excel("ng6_MCNP_spectrum.xlsx").to_numpy().T
# polyspectrum[0]*=1e-9

lammono = np.array([4.4e-10])
plammono = np.array([1])


# lampoly, plampoly = polyspectrum.copy()


h = 0.27*pi / (Nbc*4.4e-10)
# hpoly = 0.2*pi / (Nbc*5e-10)

phimono = Nbc*lammono*h

# phipoly = Nbc*lampoly*hpoly


camsize = 5e-4
slitx = 500e-6
# slitxrange = np.linspace(180,300,13)*1e-6
# slitxpolyrange = np.linspace(450,600,16)*1e-6


slity = 500e-6
xbin = 1e-6
ybin = 1e-6
res = 100e-6
# respoly = 150e-6

# Ioff = np.linspace(-5000,5000,10)



G1sdmono = {"px":px, "py":py,"L1": np.array([L1mono]), "phi":phimono}
G1sdmono["L2"] = np.linspace(11,16,1)*1e-3 


G2sdmono = {"px":px, "py":py, "L1": G1sdmono["L1"] +  G1sdmono["L2"], "phi":phimono}
G2sdmono["L2"] = Lmono - (G1sdmono["L1"] + G1sdmono["L2"])




# G1sdpoly = {"p":p, "L1": np.array([L1poly]), "phi":phipoly}
# G1sdpoly["L2"] = np.linspace(7,16,10)*1e-3 + d0["poly"]


# G2sdpoly = {"p":p, "L1": G1sdpoly["L1"] +  G1sdpoly["L2"], "phi":phipoly}
# G2sdpoly["L2"] = Lpoly - (G1sdpoly["L1"] + G1sdpoly["L2"])


simaptsmono = {"G1":RectGrating2D(G1sdmono),"G2":RectGrating2D(G2sdmono)}
siminitmono = {"lam":lammono, "plam": plammono, "L":Lmono,"d":G1sdmono["L2"],"mt":mt, "x": np.linspace(0,camsize,int(camsize/xbin)),\
               "y": np.linspace(0,camsize,int(camsize/ybin)),"x0" : x0,"xbin":xbin,"ybin":ybin,"pos":np.array([[0],[0],[1]]),\
               "camsize" : camsize,"convmode" : "valid", "slitx": slitx, "slity" : slity,"res": res, "batches" : 1000}

# simaptspoly = {"G1":RectGrating(G1sdpoly),"G2":RectGrating(G2sdpoly)}
# siminitpoly = {"lam":lampoly, "plam": plampoly, "L":Lpoly,"d":G1sdpoly["L2"],"mt":mt, "x": np.linspace(0,camsize,int(camsize/xbin)),"x0" : x0,"xbin":xbin,"ybin":ybin,\
#            "pos":np.array([[0],[1]]) ,"camsize" : camsize ,"convmode" : "valid", "slitx": slitxpoly, "slity" : slity, "res": respoly, "batches" : 2000}




In [ ]:
t0 = time()

sim2d = PGMI(simaptsmono, siminitmono)
sim2d.propagate_to("G1")
sim2d.propagate_to("G2")
sim2d.generate_after("G2")

print(time() - t0, "seconds")


In [ ]:
fig,ax = plt.subplots(figsize = (12,12))
ax.imshow(raw2dtest)
plt.show()
plt.close(fig)